In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import soundfile
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split

# all emotions on RAVDESS dataset
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# we allow only these emotions
AVAILABLE_EMOTIONS = {
    "angry",
    "sad",
    "neutral",
    "happy"
}

In [4]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [5]:
def load_data(test_size=0.2):
    X, y = [], []
    try :
      for file in glob.glob("/content/drive/My Drive/wav/Actor_*/*.wav"):
      #for file in glob.glob("/content/drive/My Drive/wav/Actor_*/*.wav"):
          # get the base name of the audio file
          basename = os.path.basename(file)
          print(basename)
          # get the emotion label
          emotion = int2emotion[basename.split("-")[2]]
          # we allow only AVAILABLE_EMOTIONS we set
          if emotion not in AVAILABLE_EMOTIONS:
              continue
          # extract speech features
          features = extract_feature(file, mfcc=True, chroma=True, mel=True)
          # add to data
          X.append(features)
          y.append(emotion)
    except :
         pass
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)

In [6]:
X_train, X_test, y_train, y_test = load_data(test_size=0.25)
# print some details
# number of samples in training data
print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])
# number of features used
# this is a vector of features extracted 
# using utils.extract_features() method
print("[+] Number of features:", X_train.shape[1])

03-01-07-02-01-01-22.wav
03-01-07-01-01-02-22.wav
03-01-07-01-02-01-22.wav
03-01-03-02-01-01-22.wav
03-01-03-01-02-01-22.wav
03-01-05-01-01-01-22.wav
03-01-02-02-02-02-22.wav
03-01-05-01-02-02-22.wav
03-01-02-01-01-02-22.wav
03-01-06-01-01-02-22.wav
03-01-05-02-02-02-22.wav
03-01-04-01-01-02-22.wav
03-01-03-02-02-02-22.wav
03-01-06-02-02-01-22.wav
03-01-04-02-02-01-22.wav
03-01-05-02-02-01-22.wav
03-01-06-02-02-02-22.wav
03-01-02-01-02-01-22.wav
03-01-02-02-01-02-22.wav
03-01-07-01-02-02-22.wav
03-01-06-01-02-02-22.wav
03-01-05-02-01-02-22.wav
03-01-04-01-01-01-22.wav
03-01-03-01-01-01-22.wav
03-01-02-01-01-01-22.wav
03-01-01-01-02-02-22.wav
03-01-06-01-02-01-22.wav
03-01-07-02-02-01-22.wav
03-01-06-01-01-01-22.wav
03-01-07-01-01-01-22.wav
03-01-06-02-01-01-22.wav
03-01-04-02-01-01-22.wav
03-01-06-02-01-02-22.wav
03-01-03-01-01-02-22.wav
03-01-01-01-01-01-22.wav
03-01-01-01-01-02-22.wav
03-01-05-02-01-01-22.wav
03-01-04-02-02-02-22.wav
03-01-03-02-01-02-22.wav
03-01-03-02-02-01-22.wav


**DECISION TREE**

In [7]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 

print(accuracy_score(y_true=y_test,y_pred=dtree_predictions))
print(classification_report(y_test,dtree_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, dtree_predictions) )

0.5263157894736842
              precision    recall  f1-score   support

       angry       0.33      0.20      0.25         5
       happy       0.38      0.60      0.46         5
     neutral       1.00      0.33      0.50         3
         sad       0.71      0.83      0.77         6

    accuracy                           0.53        19
   macro avg       0.61      0.49      0.50        19
weighted avg       0.57      0.53      0.51        19

[[1 4 0 0]
 [2 3 0 0]
 [0 0 1 2]
 [0 1 0 5]]


In [8]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


dtree_model = DecisionTreeClassifier(max_depth = 6).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 

print(accuracy_score(y_true=y_test,y_pred=dtree_predictions))
print(classification_report(y_test,dtree_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, dtree_predictions) )

0.5789473684210527
              precision    recall  f1-score   support

       angry       0.40      0.40      0.40         5
       happy       0.43      0.60      0.50         5
     neutral       0.75      1.00      0.86         3
         sad       1.00      0.50      0.67         6

    accuracy                           0.58        19
   macro avg       0.64      0.62      0.61        19
weighted avg       0.65      0.58      0.58        19

[[2 3 0 0]
 [2 3 0 0]
 [0 0 3 0]
 [1 1 1 3]]


DT

In [9]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


dtree_model = DecisionTreeClassifier(max_depth = 9,random_state=0).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 

print(accuracy_score(y_true=y_test,y_pred=dtree_predictions))
print(classification_report(y_test,dtree_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, dtree_predictions) )

0.5263157894736842
              precision    recall  f1-score   support

       angry       0.40      0.40      0.40         5
       happy       0.43      0.60      0.50         5
     neutral       0.60      1.00      0.75         3
         sad       1.00      0.33      0.50         6

    accuracy                           0.53        19
   macro avg       0.61      0.58      0.54        19
weighted avg       0.63      0.53      0.51        19

[[2 3 0 0]
 [2 3 0 0]
 [0 0 3 0]
 [1 1 2 2]]


**SUPPORT VECTOR MACHINE**

In [10]:
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train) 
svm_predictions = svm_model_linear.predict(X_test) 


print(accuracy_score(y_true=y_test,y_pred=svm_predictions))
print(classification_report(y_test,svm_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, svm_predictions) )

0.5263157894736842
              precision    recall  f1-score   support

       angry       0.50      0.80      0.62         5
       happy       0.50      0.60      0.55         5
     neutral       0.67      0.67      0.67         3
         sad       0.50      0.17      0.25         6

    accuracy                           0.53        19
   macro avg       0.54      0.56      0.52        19
weighted avg       0.53      0.53      0.49        19

[[4 1 0 0]
 [2 3 0 0]
 [0 0 2 1]
 [2 2 1 1]]


**RANDOM FOREST**

In [11]:
from sklearn.ensemble import RandomForestClassifier
  
 # create regressor object 
classifier = RandomForestClassifier(n_estimators = 100, random_state = 0) 
  
# fit the regressor with x and y data 
classifier.fit(X_train, y_train)   

c_p = classifier.predict(X_test) 


print(accuracy_score(y_true=y_test,y_pred=c_p))
print(classification_report(y_test,c_p)) 
# creating a confusion matrix 
print(confusion_matrix(y_test,c_p) )

0.5263157894736842
              precision    recall  f1-score   support

       angry       0.60      0.60      0.60         5
       happy       0.43      0.60      0.50         5
     neutral       0.50      0.33      0.40         3
         sad       0.60      0.50      0.55         6

    accuracy                           0.53        19
   macro avg       0.53      0.51      0.51        19
weighted avg       0.54      0.53      0.52        19

[[3 2 0 0]
 [2 3 0 0]
 [0 0 1 2]
 [0 2 1 3]]
